In [ ]:
import ee
import datetime
import pandas as pd

ee.Authenticate()
ee.Initialize()

def get_land_cover(lat, lon, date):
    # Convert the date string to a datetime object
    date = datetime.datetime.strptime(date, '%Y-%m-%d')

    # Check if the date is past 2021-01-01 and set it to 2021-01-01 if so, 2021-01-01 is the last date in the land cover dataset
    latest_date = datetime.datetime(2021, 1, 1)
    if date > latest_date:
        date = latest_date

    point = ee.Geometry.Point([lon, lat])

    # Load the MODIS Land Cover dataset
    dataset = ee.ImageCollection('MODIS/006/MCD12Q1') \
        .filterDate(ee.Date(date).advance(-1, 'year'), ee.Date(date)) \
        .sort('system:time_start', opt_ascending=False) \
        .first()

    if dataset is None or dataset.getInfo() == {}:
        return None

    # Select the land cover value
    land_cover = dataset.select('LC_Type1')

    # Get the land cover value for the point
    land_cover_samples = land_cover.sample(point, 500)

    if land_cover_samples.size().getInfo() == 0:
        return None

    land_cover_value = land_cover_samples.first().get('LC_Type1').getInfo()

    return land_cover_value


import csv
import pandas as pd

# Function to save the results to a CSV file
def save_to_csv(filename, data):
    with open(filename, mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Latitude', 'Longitude', 'Date', 'Land Cover'])

        for row in data:
            writer.writerow(row)

# Read the CSV file
df = pd.read_csv("mapdata_wildfires.csv")

coordinates_and_dates = []

for _, row in df.iterrows():
    lat = row['incident_latitude']
    lon = row['incident_longitude']
    date_created = row['incident_dateonly_created']

    # Create a tuple and append it to the list
    coordinates_and_dates.append((lat, lon, date_created))


results = []

# Iterate through the coordinate and date pairs and get the land cover values
for lat, lon, date in coordinates_and_dates:
    land_cover = get_land_cover(lat, lon, date)
    print(f'Latitude: {lat}, Longitude: {lon}, Date: {date}, Land Cover: {land_cover}')
    results.append((lat, lon, date, land_cover))

# Save the results to a CSV file
save_to_csv('land_cover_data_wildfires.csv', results)


In [ ]:
print("The results were saved to a CSV file.")